In [5]:
cd("/home/gridsan/aramadhan/tmp")
pwd()

"/home/gridsan/aramadhan/tmp"

In [19]:
# Source: https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/src/trials.jl
function prettytime(t)
    if t < 1e3
        value, units = t, "ns"
    elseif t < 1e6
        value, units = t / 1e3, "μs"
    elseif t < 1e9
        value, units = t / 1e6, "ms"
    else
        s = t / 1e9
        if s < 60
            value, units = s, "s"
        else
            value, units = (s / 60), "min"
        end
    end
    return string(@sprintf("%.3f", value), " ", units)
end

prettytime (generic function with 1 method)

In [15]:
# https://github.com/IainNZ/Humanize.jl/blob/master/src/Humanize.jl

import Printf: @sprintf

const SUFFIXES = Dict(
    :dec => [" B", " kB", " MB", " GB", " TB", " PB", " EB", " ZB", " YB"],
    :bin => [" B", " KiB", " MiB", " GiB", " TiB", " PiB", " EiB", " ZiB", " YiB"],
    :gnu => ["", "K", "M", "G", "T", "P", "E", "Z", "Y"]
)
const BASES = Dict(:dec => 1000, :bin => 1024, :gnu => 1024)



"""
    datasize(value::Number; style=:dec, format="%.1f")
Format a number of bytes in a human-friendly format (eg. 10 kB).
    style=:dec - default, decimal suffixes (kB, MB), base 10^3
    style=:bin - binary suffixes (KiB, MiB), base 2^10
    style=:gnu - GNU-style (ls -sh style, K, M), base 2^10
"""
function datasize(value::Number; style=:dec, format="%.1f")::String
    suffix = SUFFIXES[style]
    base = float(BASES[style])
    bytes = float(value)
    unit = base
    biggest_suffix = suffix[1]
    for power in 1:length(suffix)
        unit = base^power
        biggest_suffix = suffix[power]
        bytes < unit && break
    end
    value = base * bytes / unit
    format = "$format%s"
    return @eval @sprintf($format, $value, $biggest_suffix)
end

datasize

In [14]:
using Printf
using NetCDF

In [48]:
N = 256
A = rand(N, N, N)

for cl in 0:9
    tic = time_ns()
    
    filename = "compress" * string(cl) * ".nc"
    varname  = "rands"
    attribs  = Dict("units"   => "m/s")

    nccreate(filename, varname,
             "x1", collect(1:N), Dict("units"=>"m"),
             "x2", collect(1:N), Dict("units"=>"m"),
             "x3", collect(1:N), Dict("units"=>"m"),
             atts=attribs, chunksize=(1,16,256), compress=cl)
    ncwrite(A, filename, varname)
    ncclose(filename)
    
    toc = time_ns()
    
    ts = prettytime(toc - tic)
    fs = datasize(filesize(filename); style=:bin, format="%.3f")
    println("Compression level $cl: $ts $fs")
end

Compression level 0: 2.996 s 111.172 MiB
Compression level 1: 3.009 s 111.172 MiB
Compression level 2: 3.058 s 111.172 MiB
Compression level 3: 3.047 s 111.172 MiB
Compression level 4: 3.069 s 111.172 MiB
Compression level 5: 3.027 s 111.172 MiB
Compression level 6: 3.029 s 111.172 MiB
Compression level 7: 3.046 s 111.172 MiB
Compression level 8: 3.016 s 111.172 MiB
Compression level 9: 2.969 s 111.172 MiB


In [21]:
ncclose(filename)

65536